# MaveenaData SQLite3 Setup

In [1]:
import pandas as pd
import sqlite3, os
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
pwd

'/Users/jonathankim/Documents/MaveenaData/mavdata_sqlite'

In [4]:
Data_PATH = 'Data/'
Database_PATH = 'Databases/'
Object_PATH = 'Objects/'

In [10]:
def create_DB(db_name, table_dict, rewrite_db=True):
    '''
    Create sqlite3 database.
    
    Parameters:
        > db_name => str:
            Name of database

        > table_dict => dict:
            key (str): table name
            value (df): table dataframe
            
        > rewrite_db ==> Bool (Default=True):
            If True, then db file will be overwritten if exists.
            If False, then error will be raised.
    '''
    import sqlite3, os
    
    if rewrite_db == True:
        # Delete db if exists.
        try:
            os.remove(db_name)
        except OSError:
            pass

    with sqlite3.connect(db_name) as c:

        # Creating Tables.
        print("Creating Tables...")
        for k,v in table_dict.items():
            print('   ',k)
            v.to_sql(k,con=c)
            
        print("Vacuuming...")
        c.execute("vacuum")
        
        print("Completed")

## admission.sqlite3

### Full Dataset

In [25]:
# Bring in excel files.
retail_1 = pd.read_excel(Data_PATH+"online_retail_II.xlsx",sheet_name=0)
retail_2 = pd.read_excel(Data_PATH+"online_retail_II.xlsx",sheet_name=1)

In [26]:
# Concat them.
retail_all = pd.concat([retail_1,retail_2])

In [27]:
# New Column with Month Start for InvoiceDate.
retail_all['InvoiceDate_MonthStart'] = retail_all['InvoiceDate'].to_numpy().astype('datetime64[M]')
           

In [28]:
# No spaces in column names.
retail_all.columns = [c.replace(' ','') for c in retail_all.columns]

In [29]:
print(retail_all.shape)
retail_all.head()

(1067371, 9)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,InvoiceDate_MonthStart
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-12-01
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-12-01
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-12-01


### Split Dataset

Following tables:
- Transaction Table
- Product Table
- Customer Table

#### Transaction Table

In [31]:
transaction_table = retail_all[['Invoice','StockCode','Quantity','InvoiceDate','Price','CustomerID']]


In [32]:
transaction_table

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID
0,489434,85048,12,2009-12-01 07:45:00,6.95,13085.0
1,489434,79323P,12,2009-12-01 07:45:00,6.75,13085.0
2,489434,79323W,12,2009-12-01 07:45:00,6.75,13085.0
3,489434,22041,48,2009-12-01 07:45:00,2.10,13085.0
4,489434,21232,24,2009-12-01 07:45:00,1.25,13085.0
...,...,...,...,...,...,...
541905,581587,22899,6,2011-12-09 12:50:00,2.10,12680.0
541906,581587,23254,4,2011-12-09 12:50:00,4.15,12680.0
541907,581587,23255,4,2011-12-09 12:50:00,4.15,12680.0
541908,581587,22138,3,2011-12-09 12:50:00,4.95,12680.0


#### Product Table

In [33]:
# Group by StockCode and Description
product_table = pd.DataFrame(retail_all.groupby(['StockCode','Description'])\
                             .agg({'Invoice':'count'}))\
                .reset_index()\
                .rename(columns={'Invoice':'Count'})\
                .sort_values('Count',ascending=False)

# Only select StockCode with the highest count to avoid duplicates.
product_table = product_table.groupby('StockCode').head(1).reset_index(drop=True).drop(columns=['Count'])


In [34]:
print(product_table.shape)
product_table.head()

(4950, 2)


,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,22423,REGENCY CAKESTAND 3 TIER
2,85099B,JUMBO BAG RED RETROSPOT
3,84879,ASSORTED COLOUR BIRD ORNAMENT
4,47566,PARTY BUNTING


In [35]:
# Check if there are any duplicates.
any(product_table.duplicated('StockCode'))

False

#### Customer Table

In [36]:
# Group by Customer ID, Country, and InvoiceDate.
customer_table = pd.DataFrame(retail_all.groupby(['CustomerID','Country','InvoiceDate_MonthStart']).size()).reset_index()\
                    .rename(columns={0:'Count'}).sort_values('InvoiceDate_MonthStart',ascending=True).reset_index(drop=True)
                                                                  
# Dedupe by only including unique Customer ID and Country.
customer_table = customer_table.drop_duplicates(subset=['CustomerID','Country'])

# Rename column.
customer_table = customer_table.rename(columns={'InvoiceDate_MonthStart':'FirstTransaction_Month'}).drop(columns=['Count'])


In [37]:
print(customer_table.shape)
customer_table.head()

(5955, 3)


,CustomerID,Country,FirstTransaction_Month
0,12346.0,United Kingdom,2009-12-01
1,14828.0,United Kingdom,2009-12-01
2,14831.0,United Kingdom,2009-12-01
3,16987.0,United Kingdom,2009-12-01
4,17998.0,United Kingdom,2009-12-01


In [38]:
# Check if there are any duplicates.
any(customer_table.duplicated(['CustomerID','Country']))

False

### Export to Pickle

In [40]:
# Export to pickle.
transaction_table.to_pickle(Object_PATH+'admission_transaction_table.pkl')
product_table.to_pickle(Object_PATH+'admission_product_table.pkl')
customer_table.to_pickle(Object_PATH+'admission_customer_table.pkl')

In [37]:
pwd

'/Users/jonathankim/Documents/MaveenaData/mavdata_sqlite/Databases'

In [36]:
Object_PATH

'Objects/'

In [6]:
# Import pickle objects.
transaction_table = pd.read_pickle(Object_PATH+'admission_transaction_table.pkl')
product_table = pd.read_pickle(Object_PATH+'admission_product_table.pkl')
customer_table = pd.read_pickle(Object_PATH+'admission_customer_table.pkl')

### Export to db

In [7]:
table_dict = {'transaction_table':transaction_table,
              'product_table':product_table,
              'customer_table':customer_table}

In [12]:
create_DB(db_name=Database_PATH+"admission.sqlite3",
          table_dict=table_dict)

Creating Tables...
    transaction_table
    product_table
    customer_table
Vacuuming...
Completed


## Reference (WDI)

In [ ]:
def data():
    frame = pd.read_excel(admission_PATH+"Online Retail.xlsx")
    frame = proc_tbl(frame)
    frame = frame.drop(columns=["country_name", "indicator_name"])

    frame = frame.set_index(["indicator_code", "country_code"], verify_integrity=True)
    # frame = frame.stack()
    frame = frame.stack().rename_axis(index={None: "year"}).rename("value")
    # convert year to int
    frame.index = frame.index.set_levels(frame.index.levels[-1].astype(int), level=-1)
    return frame

In [8]:
def table_name(csv: str):
    return csv.replace("-","_").replace("WDI", "wdi_").lower()

In [12]:

with sqlite3.connect("wdi.sqlite3") as c:
    print("reading data")
    d = data()
    print("saving data")
    d.to_sql(table_name("WDIData"), con=c)
    for csv in ['WDI_CSV/'+c for c in 
                ["WDICountry",
                 "WDICountry-Series",
                 "WDIFootNote",
                 "WDISeries",
                 "WDISeries-Time"]
    ]:
        tblname = table_name(csv)
        print(f"{csv} -> {tblname}")
        proc_tbl(pd.read_csv(f"{csv}.csv")).to_sql(tblname, con=c, index=False)
        # c.execute(f"create index on {tblname}")
    print("vacuuming")
    c.execute("vacuum")

reading data
saving data
WDI_CSV/WDICountry -> wdi__csv/wdi_country
WDI_CSV/WDICountry-Series -> wdi__csv/wdi_country_series
WDI_CSV/WDIFootNote -> wdi__csv/wdi_footnote
WDI_CSV/WDISeries -> wdi__csv/wdi_series
WDI_CSV/WDISeries-Time -> wdi__csv/wdi_series_time
vacuuming
